# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import re
import joblib
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import pandas as pd
from sqlalchemy import create_engine
from classification_model.evaluation import generate_report, metrics_summary, \
                                            confusion_matrix
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package punkt to
[nltk_data]     /home/stivenlopez/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/stivenlopez/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/stivenlopez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///MessagesDisaster.db')
df = pd.read_sql_table('messages_disaster', con=engine)

cols = df.columns[4:]

X = df['message']
y = df.loc[:, cols]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text.lower())
    words = word_tokenize(text=text)
    words = [word for word in words if word not in stopwords.words('english')]
    words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
rf_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))])
gb_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                        ('clf', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))])
svm_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                         ('clf', MultiOutputClassifier(SVC()))])

In [ ]:
recall = {}
precision = {}
f1_score = {}
recall['svm'] = [cross_val_score(estimator=svm_pipeline, X=X, y=y, scoring='recall', cv=5).mean()]
recall['rf'] = [cross_val_score(estimator=rf_pipeline, X=X, y=y, scoring='recall', cv=5).mean()]
recall['gb'] = [cross_val_score(estimator=gb_pipeline, X=X, y=y, scoring='recall', cv=5).mean()]
precision['svm'] = [cross_val_score(estimator=svm_pipeline, X=X, y=y, scoring='precision', cv=5).mean()]
precision['rf'] = [cross_val_score(estimator=rf_pipeline, X=X, y=y, scoring='precision', cv=5).mean()]
precision['gb'] = [cross_val_score(estimator=gb_pipeline, X=X, y=y, scoring='precision', cv=5).mean()]
f1_score['svm'] = [cross_val_score(estimator=svm_pipeline, X=X, y=y, scoring='f1', cv=5).mean()]
f1_score['rf'] = [cross_val_score(estimator=rf_pipeline, X=X, y=y, scoring='f1', cv=5).mean()]
f1_score['gb'] = [cross_val_score(estimator=gb_pipeline, X=X, y=y, scoring='f1', cv=5).mean()]

/home/stivenlopez/PycharmProjects/Disaster-response/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of classes has to be greater than one; got 1 class

  FitFailedWarning)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.